Importing libraries

In [14]:
# Core libraries
import pandas as pd
import numpy as np
import pickle
import time
import warnings
warnings.filterwarnings('ignore', category=Warning) # Suppress convergence warnings from Gaussian Process


#learning curve for baseline model
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit



# Hyperparameter optimization
import optuna
from optuna.samplers import TPESampler

# XGBoost
import xgboost as xgb
from xgboost import plot_importance, plot_tree

# Machine Learning
from sklearn.model_selection import (
    train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, learning_curve, ShuffleSplit
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, classification_report, confusion_matrix,
    roc_curve, precision_recall_curve, log_loss
)

# Bayesian Optimization
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)

# Configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [2]:
with open('processed/extended_preprocessed.pkl', 'rb') as f:
    extended_data = pickle.load(f)

# Extract all datasets including validation
X_train_extended = extended_data['X_train']
X_val_extended = extended_data['X_val']
X_test_extended = extended_data['X_test'] 
y_train = extended_data['y_train']
y_val = extended_data['y_val']
y_test = extended_data['y_test']

print(f"\n📊 Dataset Shapes:")
print(f"Training set: {X_train_extended.shape}")
print(f"Validation set: {X_val_extended.shape}")
print(f"Test set: {X_test_extended.shape}")
print(f"Features: {X_train_extended.shape[1]}")

print(f"\n🎯 Target Distribution:")
print(f"Training - Over 2.5: {y_train.mean():.2%}")
print(f"Validation - Over 2.5: {y_val.mean():.2%}")
print(f"Test - Over 2.5: {y_test.mean():.2%}")


# Display data split summary
total_samples = len(y_train) + len(y_val) + len(y_test)
print(f"\n📈 Data Split Summary:")
print(f"Total samples: {total_samples:,}")
print(f"Training: {len(y_train):,} ({len(y_train)/total_samples:.1%})")
print(f"Validation: {len(y_val):,} ({len(y_val)/total_samples:.1%})")
print(f"Test: {len(y_test):,} ({len(y_test)/total_samples:.1%})")


📊 Dataset Shapes:
Training set: (20940, 693)
Validation set: (7277, 693)
Test set: (14376, 693)
Features: 693

🎯 Target Distribution:
Training - Over 2.5: 48.97%
Validation - Over 2.5: 49.10%
Test - Over 2.5: 51.93%

📈 Data Split Summary:
Total samples: 42,593
Training: 20,940 (49.2%)
Validation: 7,277 (17.1%)
Test: 14,376 (33.8%)


In [10]:
import re
 
def clean_feature_names(df):
    df = df.copy()
    new_cols = []
    for c in df.columns:
        s = str(c)
        # Replace forbidden chars with safe tokens
        s = s.replace("[", "(").replace("]", ")")
        s = s.replace("<", "_lt_").replace(">", "_gt_")
        new_cols.append(s)
    df.columns = new_cols
    return df
 
X_train_extended = clean_feature_names(X_train_extended)
X_val_extended   = clean_feature_names(X_val_extended)
X_test_extended  = clean_feature_names(X_test_extended)
 
print(X_train_extended.columns[:20])

Index(['league_tier', 'month', 'is_weekend', 'VCH', 'VCD', 'VCA', 'VCCH',
       'VCCD', 'VCCA', 'home_days_since_last', 'away_days_since_last',
       'home_goals_ma5', 'away_goals_ma5', 'home_conceded_ma5',
       'away_conceded_ma5', 'home_promoted', 'away_promoted', 'home_demoted',
       'away_demoted', 'home_position_pct_2019'],
      dtype='object')


In [11]:
# Function to calculate comprehensive metrics
def evaluate_model(y_true, y_pred, y_pred_proba, model_name="Model"):
    """Calculate comprehensive classification metrics"""
    
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred_proba)
    }
    
    print(f"\n📈 {model_name} Performance Metrics:")
    print("=" * 50)
    print(f"Accuracy:  {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall:    {metrics['recall']:.4f}")
    print(f"F1-Score:  {metrics['f1']:.4f}")
    print(f"ROC-AUC:   {metrics['roc_auc']:.4f}")
    
    return metrics

In [17]:
def eval_model(y_true, y_pred_proba, threshold=0.5, label=""):
    y_pred = (y_pred_proba >= threshold).astype(int)
    print(f"\n== {label} ==")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("ROC AUC :", roc_auc_score(y_true, y_pred_proba))
    print("Log loss:", log_loss(y_true, y_pred_proba))

# 1) Fit basic XGB on train, tune on val
params_ext = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "tree_method": "hist",
    "learning_rate": 0.05,
    "max_depth": 5,
    "min_child_weight": 5,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "lambda": 1.0,
    "alpha": 0.0,
    "enable_categorical": True,
}

dtrain_e = xgb.DMatrix(X_train_extended, label=y_train, enable_categorical=True)
dval_e   = xgb.DMatrix(X_val_extended,   label=y_val,   enable_categorical=True)

evals = [(dtrain_e, "train"), (dval_e, "val")]
model_basic_xgb = xgb.train(
    params_ext,
    dtrain_e,
    num_boost_round=2000,
    evals=evals,
    early_stopping_rounds=100,
    verbose_eval=100
)

# 2) Evaluate on val & test
dtest_e = xgb.DMatrix(X_test_extended, enable_categorical=True)

val_pred_e  = model_basic_xgb.predict(dval_e)
test_pred_e = model_basic_xgb.predict(dtest_e)

eval_model(y_val,  val_pred_e,  label="Basic XGB – validation")
eval_model(y_test, test_pred_e, label="Basic XGB – TEST  (A0')")

[0]	train-logloss:0.69051	val-logloss:0.69124
[100]	train-logloss:0.62849	val-logloss:0.67906
[144]	train-logloss:0.61255	val-logloss:0.68053

== Basic XGB – validation ==
Accuracy: 0.558609317026247
ROC AUC : 0.594399312034886
Log loss: 0.6805280733741573

== Basic XGB – TEST  (A0') ==
Accuracy: 0.5685169727323317
ROC AUC : 0.5984203037703659
Log loss: 0.6781895332400151


In [18]:
dtrain = xgb.DMatrix(
    X_train_extended,
    label=y_train,
    enable_categorical=True
)
dval = xgb.DMatrix(
    X_val_extended,
    label=y_val,
    enable_categorical=True
)
dtest = xgb.DMatrix(
    X_test_extended,
    label=y_test,
    enable_categorical=True
)

In [19]:
def objective(trial: optuna.Trial) -> float:
    # Hyperparameter search space
    params = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",      # we optimise logloss
        "tree_method": "hist",
        "enable_categorical": True,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 10.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),  # L2
        "alpha": trial.suggest_float("alpha", 1e-3, 10.0, log=True),    # L1
        "max_bin": trial.suggest_int("max_bin", 128, 512),
    }

    evals = [(dtrain, "train"), (dval, "val")]

    # Train with early stopping on validation set
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1500,
        evals=evals,
        early_stopping_rounds=80,
        verbose_eval=False,
    )

    # Predict on validation set
    y_val_pred = model.predict(dval, iteration_range=(0, model.best_iteration + 1))

    # Compute logloss (Optuna will minimise this)
    val_logloss = log_loss(y_val, y_val_pred)

    return val_logloss

In [21]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=150, show_progress_bar=True)

print("Best trial:")
print("  Value (logloss):", study.best_value)
print("  Params:")
for k, v in study.best_params.items():
    print(f"    {k}: {v}")

[I 2025-11-13 16:29:58,086] A new study created in memory with name: no-name-62979de1-2df7-447c-a066-05ce23c61d54
Best trial: 0. Best value: 0.675578:   1%|          | 1/150 [00:02<05:14,  2.11s/it]

[I 2025-11-13 16:30:00,193] Trial 0 finished with value: 0.6755782515932182 and parameters: {'learning_rate': 0.04446949757391796, 'max_depth': 5, 'min_child_weight': 8.340268403736681, 'subsample': 0.6720186591546414, 'colsample_bytree': 0.5171706576070318, 'gamma': 3.0768270535940747, 'lambda': 0.9939179734298893, 'alpha': 0.006549506203728982, 'max_bin': 303}. Best is trial 0 with value: 0.6755782515932182.


Best trial: 0. Best value: 0.675578:   1%|▏         | 2/150 [00:06<09:12,  3.73s/it]

[I 2025-11-13 16:30:05,060] Trial 1 finished with value: 0.679534004875699 and parameters: {'learning_rate': 0.09058524687623244, 'max_depth': 10, 'min_child_weight': 7.3670150793816775, 'subsample': 0.5377308662038277, 'colsample_bytree': 0.975085653266683, 'gamma': 0.3489756244539999, 'lambda': 7.081332981331486, 'alpha': 0.0324239354383255, 'max_bin': 400}. Best is trial 0 with value: 0.6755782515932182.


Best trial: 0. Best value: 0.675578:   2%|▏         | 3/150 [00:10<08:44,  3.57s/it]

...

[I 2025-11-13 16:38:16,621] Trial 147 finished with value: 0.6740502476245045 and parameters: {'learning_rate': 0.013101065376963202, 'max_depth': 3, 'min_child_weight': 2.267397800610718, 'subsample': 0.7510555695254234, 'colsample_bytree': 0.9737990313268028, 'gamma': 4.697497718860026, 'lambda': 0.03168913431016096, 'alpha': 0.035550095382679975, 'max_bin': 441}. Best is trial 131 with value: 0.6733408474216914.


Best trial: 131. Best value: 0.673341:  99%|█████████▉| 149/150 [08:22<00:04,  4.26s/it]

[I 2025-11-13 16:38:20,949] Trial 148 finished with value: 0.6736831245016769 and parameters: {'learning_rate': 0.01856759643952477, 'max_depth': 3, 'min_child_weight': 1.5055189596850924, 'subsample': 0.7670386456335156, 'colsample_bytree': 0.9499385475253225, 'gamma': 3.6996242388400673, 'lambda': 0.0012498221491753334, 'alpha': 0.06957627763157667, 'max_bin': 491}. Best is trial 131 with value: 0.6733408474216914.


Best trial: 131. Best value: 0.673341: 100%|██████████| 150/150 [08:27<00:00,  3.39s/it]

[I 2025-11-13 16:38:25,920] Trial 149 finished with value: 0.6738518848258299 and parameters: {'learning_rate': 0.016572270688716167, 'max_depth': 3, 'min_child_weight': 1.8694459616385974, 'subsample': 0.7832981463479244, 'colsample_bytree': 0.9574659962610559, 'gamma': 4.125700619067718, 'lambda': 0.019459030162498, 'alpha': 0.02819205570542265, 'max_bin': 485}. Best is trial 131 with value: 0.6733408474216914.
Best trial:
  Value (logloss): 0.6733408474216914
  Params:
    learning_rate: 0.020874191443998248
    max_depth: 3
    min_child_weight: 1.3153905946153215
    subsample: 0.7719709099784914
    colsample_bytree: 0.9558089088940487
    gamma: 3.9817341935881796
    lambda: 0.010957428707340578
    alpha: 0.06955143006068921
    max_bin: 471


In [ ]:
# Combine train + val for final training
X_ext_trainval = pd.concat([X_train_extended, X_val_extended], axis=0).reset_index(drop=True)
y_ext_trainval = np.concatenate([y_train, y_val])

dtrainval = xgb.DMatrix(
    X_ext_trainval,
    label=y_ext_trainval,
    enable_categorical=True
)

# Merge best params with fixed ones
best_params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "tree_method": "hist",
    "enable_categorical": True,
}
best_params.update(study.best_params)

evals_final = [(dtrainval, "trainval")]

final_model = xgb.train(
    best_params,
    dtrainval,
    num_boost_round=study.best_trial.user_attrs.get("best_iteration", 1500),
    evals=evals_final,
    early_stopping_rounds=80,
    verbose_eval=False,
)

In [ ]:
#only for train
best_params = {
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "tree_method": "hist",
    "enable_categorical": True,
}
best_params.update(study.best_params)

evals_final = [(dtrain, "train")]

final_model = xgb.train(
    best_params,
    dtrainval,
    num_boost_round=study.best_trial.user_attrs.get("best_iteration", 1500),
    evals=evals_final,
    early_stopping_rounds=80,
    verbose_eval=False,
)

In [27]:
val_pred  = final_model.predict(dval)
test_pred = final_model.predict(dtest)

eval_model(y_val,  val_pred,  label="Extended XGB (Bayes-tuned) – validation")
eval_model(y_test, test_pred, label="Extended XGB (Bayes-tuned) – TEST")


== Extended XGB (Bayes-tuned) – validation ==
Accuracy: 0.7262608217672117
ROC AUC : 0.814051261289525
Log loss: 0.5849654241776837

== Extended XGB (Bayes-tuned) – TEST ==
Accuracy: 0.5601697273233166
ROC AUC : 0.5901648972395463
Log loss: 0.6847684602960555
